In [1]:
import json
import time
import numpy as np
import pandas as pd


In [2]:
fname = "./goodreads_interactions.csv"
start = time.time()
interactions = pd.read_csv(fname)

interactions = interactions.rename(columns={"book_id":"book_id_csv"})
print(time.time() - start)



106.21423196792603


In [3]:
interactions_book_ids = set(interactions["book_id_csv"].unique())

print(min(interactions_book_ids))
print(max(interactions_book_ids))
print(interactions.shape)
# interactions1 = interactions[(interactions.is_read > 0) & (interactions.is_reviewed > 0)]
interactions = interactions[(interactions.rating > 0)]
interactions = interactions.drop(['is_reviewed', 'is_read'], axis=1)
# interactions3 = interactions[(interactions.is_read > 0) & (interactions.is_reviewed > 0) & (interactions.rating > 0)]
# print(interactions1.shape)
# print(interactions2.shape)
# print(interactions2.shape)
# print(interactions3.shape)

0
2360649
(228648342, 5)


,user_id,book_id_csv,rating
0,0,948,5
1,0,947,5
2,0,946,5
3,0,945,5
4,0,944,5
...,...,...,...
228648336,876144,367428,3
228648338,876144,23847,4
228648339,876144,23950,3
228648340,876144,374106,5


In [11]:
fname = './goodreads_books.json'
from io import StringIO
from csv import writer 

output = StringIO()
csv_writer = writer(output)
i = 0
bad = []
start = time.time()
f = open(fname)
for line in f:
    try:
        s = json.loads(line)
    except json.JSONDecodeError:
        bad.append(line)
        continue
#     if(i % 100000 == 0):
#         print(i)
    book_id = s["book_id"]
    title = s['title']
    authors = s['authors']
    average_rating = s['average_rating']
    image_url = s['image_url']
    num_pages = s['num_pages']
    publisher = s['publisher']
    publication_year = s['publication_year']
    ratings_count = s['ratings_count']
    language_code = s['language_code']
    csv_writer.writerow([book_id, title, authors, language_code, average_rating, image_url, num_pages, publisher, publication_year, ratings_count])
    i = i + 1

output.seek(0) # we need to get back to the start of the BytesIO
books = pd.read_csv(output, header=None)
books.columns = ['book_id', 'title', 'authors', 'language_code', 'average_rating', 'image_url', 'num_pages', 'publisher', 'publication_year', 'ratings_count']
f.close()
books.to_csv("books.csv")
print(time.time() - start)


152.16972398757935


In [5]:
fname = "./book_id_map.csv"
start = time.time()
book_id_map = pd.read_csv(fname)
print(time.time() - start)

0.5779259204864502


In [6]:
books_book_ids = set(books["book_id"].unique())
print(min(books_book_ids))
print(max(books_book_ids))
print(len(books_book_ids))
books.head()

NameError: name 'books' is not defined

In [36]:
interactions_book_ids = set(interactions["book_id_csv"].unique())
print(min(interactions_book_ids))
print(max(interactions_book_ids))
print(len(interactions_book_ids))
interactions.head()

0
2360649
2325541


,user_id,book_id_csv,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [63]:
book_id_map_id_csv = set(book_id_map["book_id_csv"].unique())
book_id_map_id = set(book_id_map["book_id"].unique())
print(min(book_id_map_id_csv))
print(max(book_id_map_id_csv))
print(len(book_id_map_id_csv))
print(min(book_id_map_id))
print(max(book_id_map_id))
print(len(book_id_map_id))
book_id_map.head()

0
2360649
2360650
1
36530431
2360650


,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361


In [7]:
interactions1 = pd.merge(interactions, book_id_map, on="book_id_csv")
print(len(interactions1))

104551549


In [9]:
interactions1 = interactions1.drop(['is_reviewed', 'is_read', 'book_id_csv'], axis=1)
interactions1.head()

,user_id,rating,book_id
0,0,5,12
1,201,3,12
2,339,3,12
3,683,5,12
4,1872,3,12


In [10]:
# ratings_full includes all rows for ratings > 0 even if is_reviewd flag is set to 0
interactions1.to_csv("ratings_full.csv")

In [77]:
book_id_map[book_id_map['book_id_csv'] == 947]

,book_id_csv,book_id
947,947,21


In [40]:
print(len(interactions))
print(len(interactions1))
interactions1
interactions.head()

104551549
104551549


,user_id,book_id_csv,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [110]:
# len(books['authors']) > 0
books['authors'].str.len()
# books['authors'] = books['authors'].apply(lambda x: x.strip('][').split('},'))
books['authors'] = books['authors'].apply(lambda x: [str(y) + '}' for y in x])
# books['authors'] = 
books.dtypes
str1 = books.iat[0, books.columns.get_loc('authors')]
str1 = "[{'author_id': '604031', 'role': ''}, {'author_id': '604032', 'role': ''}]"

print(str1)
res = str1.strip('][').split('},')
[str(x) + '}' for x in res]

[{'author_id': '604031', 'role': ''}, {'author_id': '604032', 'role': ''}]


["{'author_id': '604031', 'role': ''}",
 " {'author_id': '604032', 'role': ''}}"]

In [108]:
print (type(books.iat[0, books.columns.get_loc('authors')]))

<class 'list'>


In [122]:
books.iloc[2360650]["authors"][0]

"{'author_id': '4015', 'role': ''}"